<a href="https://colab.research.google.com/github/miladramzy/MCDM_DOE_UBC_ENGR_589/blob/main/Lab2_ANOVA_Block_Design_Latin_Square.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENGR 489/589
### School of Engineering - Okanagan Campus
### By Dr. Milad Ramezankhani, Dr. Abbas Milani
#### milad.ramezankhani@ubc.ca | https://miladramzy.github.io/
## Lab 2 - ANOVA - Block design - Lating Square

### Content:

* ANOVA
    * one-way
    * Block desgin
* Latin Square

In [ ]:
# Import necessary libraries
import scipy.stats as stats
import pandas as pd
import numpy as np
import scipy.stats

In [ ]:
# Template list for providing the experiment data
# Rows: Treatment (main factor)
# Columns: Observation repeats
data = [[],
        [],
        [],
        []]

In [ ]:
data = [[575,542,530,539,570],
        [565,593,590,579,610],
        [600,651,610,637,629],
        [725,700,715,685,710]]

##   
## ANOVA

In [ ]:
# Method 1: Using Scipy library (manual, not recommended)
stats.f_oneway(data[0], data[1], data[2], data[3])

F_onewayResult(statistic=66.79707321945864, pvalue=2.882865908493268e-09)

In [ ]:
# Method 2: Manual Calculation (full ANOVA table)
def Anova_OneWay(data, alpha, Blocks = False):

    data = np.array(data)
    no_of_treatments = data.shape[0] # number of rows
    blocks = data.shape[1] # number of columns
    sum_total = np.sum(data) # Total sum

    # Sum of squares
    bias = (sum_total**2)/(no_of_treatments*blocks) # sum of squares of all observations divided by sample size
    ss_total = np.sum(np.square(data))-bias # Total SS
    ss_treatments = np.sum(np.square(np.sum(data, axis=1)))/blocks-bias # Treatment SS
    if Blocks:
        ss_blocks = np.sum(np.square(np.sum(data, axis=0)))/no_of_treatments-bias # Block SS
    else:
        ss_blocks = 0
    ss_error = ss_total - ss_treatments-ss_blocks ## Error SS

    # Degrees of freedom
    df_total = no_of_treatments*blocks-1 # Total dof
    df_treatments = no_of_treatments - 1 # Treatment dof
    if Blocks:
        df_blocks = blocks - 1 # Block dof
    else:
        df_blocks = 0
    df_error = df_total - df_treatments - df_blocks # Error dof

    # Mean squares
    ms_treatments = ss_treatments/df_treatments # Treatment MS
    if Blocks:
        ms_blocks = ss_blocks/df_blocks # Block MS
    else:
        ms_blocks = 0
    ms_error = ss_error/df_error # Error MS

    # F and p-value
    F_treatments = ms_treatments/ms_error # Treatment F_0
    F_crt_onetail = stats.f.ppf(1-alpha, df_treatments,df_error) # Treatment critical F
    p_value_treatments = 1-scipy.stats.f.cdf(F_treatments, dfn=df_treatments, dfd=df_error) # Treatment P-value

    if Blocks:
        F_blocks = ms_blocks/ms_error # Block F_0
        Fcrt_blocks = stats.f.ppf(1-alpha, df_blocks,df_error) # Block critical F
        p_value_blocks = 1-scipy.stats.f.cdf(F_blocks, dfn=df_blocks, dfd=df_error) # Block P-value
    else:
        F_blocks, Fcrt_blocks, p_value_blocks = [0,0,0]


    df = pd.DataFrame([[ss_treatments, df_treatments, ms_treatments, F_treatments, p_value_treatments, F_crt_onetail],
                       [ss_blocks, df_blocks, ms_blocks, F_blocks, p_value_blocks, Fcrt_blocks],
                       [ss_error, df_error, ms_error, np.nan, np.nan, np.nan],
                       [ss_total, df_total, np.nan, np.nan, np.nan, np.nan]], columns = ['SS', 'df', 'MS', 'F', 'P-value', 'F crit'],
                     index = ['Treatment', 'Block', 'Error', 'Total'])
    return df

In [ ]:
# Example 1: One-way ANOVA
data = [[575,542,530,539,570],
        [565,593,590,579,610],
        [600,651,610,637,629],
        [725,700,715,685,710]]
alpha = 0.05
Anova_OneWay(data, alpha, Blocks = False)

,SS,df,MS,F,P-value,F crit
Treatment,66870.55,3,22290.183333,66.797073,2.882866e-09,3.238872
Block,0.00,0,0.000000,0.000000,0.000000e+00,0.000000
Error,5339.20,16,333.700000,NaN,NaN,NaN
Total,72209.75,19,NaN,NaN,NaN,NaN


In [ ]:
# Example 1: One-way ANOVA with blocking design
data = [[575,542,530,539,570],
        [565,593,590,579,610],
        [600,651,610,637,629],
        [725,700,715,685,710]]
alpha = 0.05
Anova_OneWay(data, alpha, Blocks = True)

,SS,df,MS,F,P-value,F crit
Treatment,66870.55,3,22290.183333,62.369063,1.368674e-07,3.490295
Block,1050.50,4,262.625000,0.734838,5.856737e-01,3.259167
Error,4288.70,12,357.391667,NaN,NaN,NaN
Total,72209.75,19,NaN,NaN,NaN,NaN


##   
## Latin Square  

In [ ]:
def LatinSquare(data, data_sorted, alpha):
    data = np.array(data)
    rows = data.shape[0]
    sum_total = np.sum(data)

    # Sum of squaares
    bias = (sum_total**2)/(rows**2)
    ss_total = np.sum(np.square(data))-bias
    ss_treatment = np.sum(np.square(np.sum(data_sorted, axis=1)))/rows - bias
    ss_nuisance1 = np.sum(np.square(np.sum(data, axis=1)))/rows - bias
    ss_nuisance2 = np.sum(np.square(np.sum(data, axis=0)))/rows - bias
    ss_error = ss_total - (ss_treatment + ss_nuisance1 + ss_nuisance2)

    # degree of freedom
    df_total = rows**2 - 1
    df_treatment = rows - 1
    df_nuisance1 = rows - 1
    df_nuisance2 = rows - 1
    df_error = (rows - 2)*(rows - 1)

    # Mean squares
    ms_treatment = ss_treatment/df_treatment
    ms_nuisance1 = ss_nuisance1/df_nuisance1
    ms_nuisance2 = ss_nuisance2/df_nuisance2
    ms_error = ss_error/df_error

    # F and P-values
    F_treatment = ms_treatment/ms_error
    F_critical = stats.f.ppf(1-alpha, df_treatment,df_error) # Treatment critical F
    p_value_treatments = 1-scipy.stats.f.cdf(F_treatment, df_treatment, df_error)

    F_nuisance1 = ms_nuisance1/ms_error
    Fcritical_n1 = stats.f.ppf(1-alpha, df_nuisance1,df_error)
    p_value_n1 = 1-scipy.stats.f.cdf(F_nuisance1, df_nuisance1, df_error)

    F_nuisance2 = ms_nuisance2/ms_error
    Fcritical_n2 = stats.f.ppf(1-alpha, df_nuisance2,df_error)
    p_value_n2 = 1-scipy.stats.f.cdf(F_nuisance2, df_nuisance1, df_error)


    df = pd.DataFrame([[ss_treatment, df_treatment, ms_treatment, F_treatment, p_value_treatments, F_critical],
                       [ss_nuisance1, df_nuisance1, ms_nuisance1, F_nuisance1, p_value_n1, Fcritical_n1],
                       [ss_nuisance2, df_nuisance2, ms_nuisance2, F_nuisance2, p_value_n2, Fcritical_n2],
                       [ss_error, df_error, ms_error, np.nan, np.nan, np.nan],
                       [ss_total, df_total, np.nan, np.nan, np.nan, np.nan]], columns = ['SS', 'df', 'MS', 'F', 'P-value', 'F crit'],
                     index = ['Treatment', 'Nuisance 1', 'Nuisance 2', 'Error', 'Total'])
    
    return df

In [ ]:
data = [[24,20,19],
        [17,24,30],
        [18,38,26]]
# Data_sorted example: 
#[[A,A,A],
# [B,B,B],
# [C,C,C]]
data_sorted = [[24,38,30],
               [17,20,26],
               [18,24,19]]
alpha = 0.05
LatinSquare(data, data_sorted, alpha)

,SS,df,MS,F,P-value,F crit
Treatment,200.666667,2,100.333333,25.083333,0.038339,19.0
Nuisance 1,60.666667,2,30.333333,7.583333,0.116505,19.0
Nuisance 2,92.666667,2,46.333333,11.583333,0.079470,19.0
Error,8.000000,2,4.000000,NaN,NaN,NaN
Total,362.000000,8,NaN,NaN,NaN,NaN


## End of Lab 2